In [ ]:
import googlemaps
import pandas as pd
import time
import os
from datetime import datetime

class BusinessLocator:
    def __init__(self, api_key):
        """
        Initialize the BusinessLocator with Google Maps API key
        """
        # Initialize Google Maps client. The 'googlemaps' library automatically
        # handles the underlying API version when calling methods like find_place
        # and place, especially when specific field names (camelCase for new API)
        # or input_type are provided.
        self.gmaps = googlemaps.Client(key=api_key)
        
        # Enhanced business types - added Jio and Vi service centers
        self.business_types = {
            'Airtel Service Centers': ['Airtel store', 'Airtel service center', 'Airtel outlet', 'Airtel recharge center'],
            'Jio Service Centers': ['Jio store', 'Jio service center', 'Jio outlet', 'Jio digital store', 'Reliance Jio store'],
            'Vi Service Centers': ['Vi store', 'Vi service center', 'Vodafone Idea store', 'VI outlet', 'Vodafone store', 'Idea store'],
            'Gold Showrooms': ['gold jewellery shop', 'gold showroom', 'jewellery store', 'gold jewelry', 'precious metals'],
            'Car Showrooms': ['car showroom', 'car dealer', 'automobile showroom', 'auto dealer', 'vehicle showroom'],
            'Shopping Malls': ['shopping mall', 'shopping center', 'mall', 'shopping complex', 'retail plaza'],
            'Luxury Fashion Stores': ['luxury boutique', 'designer store', 'high-end fashion', 'premium fashion', 'designer clothing'],
            'Department Stores': ['department store', 'luxury department store', 'premium retail store', 'multi-brand store'],
            'Luxury Brand Stores': ['luxury store', 'premium brand', 'designer brand store', 'high-end retail', 'luxury boutique'],
            'Banks & ATMs': ['bank', 'ATM', 'banking center', 'financial services', 'private bank'],
            'Insurance Companies': ['insurance office', 'insurance company', 'life insurance', 'general insurance', 'insurance agency'],
            'Real Estate Offices': ['real estate office', 'property dealer', 'real estate agent', 'property consultant', 'real estate firm'],
            'Premium Hotels': ['luxury hotel', '5 star hotel', '4 star hotel', 'premium hotel', 'business hotel'],
            'Restaurants & Fine Dining': ['fine dining restaurant', 'premium restaurant', 'multi-cuisine restaurant', 'luxury restaurant'],
            'Beauty & Wellness Centers': ['beauty salon', 'spa', 'wellness center', 'beauty parlor', 'premium salon']
        }
    
    def get_detailed_place_info(self, place_id):
        """
        Get detailed information for a specific place including phone and website.
        Uses Places API (New) field mask for efficiency and future compatibility.
        """
        try:
            # Use Places API (New) field names (camelCase) for requesting specific details.
            # The 'fields' parameter in googlemaps.Client.place() is implicitly used as a fieldMask
            # for the newer Places API if the underlying API is enabled for the project.
            place_details = self.gmaps.place(
                place_id=place_id,
                fields=[
                    'name', 'formattedAddress', 'formattedPhoneNumber', 'website',
                    'rating', 'userRatingsTotal', 'priceLevel', 'businessStatus',
                    'openingHours.weekdayDescriptions', 'types', 'geometry.location' # Ensure geometry.location is requested for lat/lng
                ]
            )
            
            # The 'place' method returns a dictionary with 'result' and 'html_attributions'
            # Check for 'result' in the response directly
            if 'result' in place_details:
                return place_details['result']
            else:
                # Handle cases where 'result' might be missing but status is OK, or status is not OK
                print(f"    ⚠️ No detailed result found for place {place_id} (Status: {place_details.get('status', 'N/A')})")
                return None
                
        except Exception as e:
            print(f"    ⚠️ Error getting details for place {place_id}: {str(e)[:100]}...")
            return None
    
    def search_places(self, city, business_type, search_terms):
        """
        Enhanced search that fetches detailed place information including phone and website.
        Uses Places API (New) 'find_place_from_text' for initial search.
        """
        places_data = []
        seen_places = set()  # Track duplicates by place_id
        
        for term in search_terms:
            try:
                # Use find_place_from_text (part of Places API New) for initial search.
                # 'input_type' is required for find_place.
                # 'fields' here are for the initial find_place response, often basic info like place_id.
                # Request 'place_id' and 'name' for candidate identification, and 'geometry.location'
                # for initial lat/lng if available without a detailed call.
                places_result = self.gmaps.find_place(
                    input=f"{term} in {city}",
                    input_type='textquery',
                    fields=['place_id', 'displayName', 'location'] # 'displayName' for name, 'location' for lat/lng in new API
                )
                
                # The find_place method returns 'candidates' not 'results' for the places found.
                if 'candidates' not in places_result or not places_result['candidates']:
                    print(f"    No candidates found for '{term} in {city}'.")
                    continue

                for place in places_result['candidates']:
                    place_id = place.get('place_id')
                    
                    if not place_id or place_id in seen_places:
                        continue
                    
                    seen_places.add(place_id)
                    
                    # Use 'displayName' for the name from the find_place response
                    place_name = place.get('displayName', {}).get('text', 'Unknown')
                    print(f"    🔍 Getting details for: {place_name}")
                    
                    # Get detailed information using the new method which uses field mask
                    detailed_info = self.get_detailed_place_info(place_id)
                    
                    if detailed_info:
                        # Extract phone number and website - using correct field names (camelCase from new API)
                        phone = detailed_info.get('formattedPhoneNumber', '').strip()
                        website = detailed_info.get('website', '').strip()
                        opening_hours = self._format_opening_hours(detailed_info.get('openingHours')) # 'openingHours' is camelCase
                        
                        # Only include places with meaningful data (skip if all key fields are empty)
                        # We use 'formattedAddress' for a more robust check in the new API context.
                        if not phone and not website and not detailed_info.get('rating') and not detailed_info.get('formattedAddress'):
                            continue
                        
                        # Extract location, handling potential absence gracefully
                        latitude = detailed_info.get('geometry', {}).get('location', {}).get('latitude')
                        longitude = detailed_info.get('geometry', {}).get('location', {}).get('longitude')

                        # Skip if essential location data is missing
                        if latitude is None or longitude is None:
                            print(f"    Skipping {place_name}: Missing latitude/longitude.")
                            continue

                        place_details = {
                            'Business_Type': business_type,
                            'Name': detailed_info.get('name', '').strip(), # Original name field is still 'name' in detailed response
                            'Address': detailed_info.get('formattedAddress', '').strip(), # camelCase
                            'Phone': phone if phone else '',
                            'Website': website if website else '',
                            'Latitude': latitude,
                            'Longitude': longitude,
                            'Rating': detailed_info.get('rating', ''),
                            'Total_Ratings': detailed_info.get('userRatingsTotal', ''), # camelCase
                            'Price_Level': self._get_price_level(detailed_info.get('priceLevel')), # camelCase
                            'Business_Status': detailed_info.get('businessStatus', ''), # camelCase
                            'Types': ', '.join(detailed_info.get('types', [])), # camelCase
                            'Place_ID': place_id,
                            'City': city,
                            'Search_Term': term
                        }
                        
                        places_data.append(place_details)
                        print(f"    ✅ Added: {detailed_info.get('name', 'Unknown')}")
                    
                    # Add delay between detailed API calls to avoid rate limiting
                    time.sleep(0.5)
                
                # Enhanced delay between search terms
                time.sleep(1.0)
                
            except googlemaps.exceptions.ApiError as e:
                print(f"  ❌ API Error searching for {term}: {e}")
                continue
            except Exception as e:
                print(f"  ⚠️  Error searching for {term}: {str(e)[:100]}...")
                continue
        
        return places_data
    
    def _format_opening_hours(self, opening_hours):
        """
        Format opening hours information.
        Expects 'openingHours' structure from Places API (New), which contains 'weekdayDescriptions'.
        """
        if not opening_hours:
            return ''
        
        # In Places API (New), human-readable opening hours are typically in 'weekdayDescriptions'
        if 'weekdayDescriptions' in opening_hours:
            return '; '.join(opening_hours['weekdayDescriptions'])
        
        return ''
    
    def _get_price_level(self, price_level):
        """Convert numeric price level to descriptive text"""
        if price_level is None:
            return ''
        
        price_map = {
            0: 'Free',
            1: 'Inexpensive',
            2: 'Moderate',
            3: 'Expensive',
            4: 'Very Expensive'
        }
        return price_map.get(price_level, '')
    
    def get_all_businesses(self, city):
        """
        Get all business types for a given city with enhanced progress tracking
        """
        all_data = []
        
        print(f"\n🔍 Searching for businesses in {city}...")
        print(f"📊 Total categories to search: {len(self.business_types)}")
        
        for idx, (business_type, search_terms) in enumerate(self.business_types.items(), 1):
            print(f"  📍 [{idx}/{len(self.business_types)}] Searching for {business_type}...")
            
            business_data = self.search_places(city, business_type, search_terms)
            all_data.extend(business_data)
            
            print(f"      ✅ Found {len(business_data)} {business_type.lower()}")
            
            # Add small delay between categories
            if idx < len(self.business_types):
                time.sleep(1.0)
        
        print(f"\n🎯 Total businesses found: {len(all_data)}")
        return all_data
    
    def clean_data(self, data):
        """
        Clean data by removing rows with too many empty values.
        Adjusted to use 'Address' instead of 'Opening_Hours' for key field check,
        as Opening_Hours might not always be present or critical for basic entry.
        """
        if not data:
            return data
        
        cleaned_data = []
        for item in data:
            # Count non-empty values (excluding location data which should always be present)
            non_empty_count = 0
            
            # Check key fields. 'Address' is usually a reliable indicator of useful data.
            key_fields = ['Name', 'Address', 'Phone', 'Website', 'Rating'] # Removed 'Opening_Hours' as it's less critical for basic filtering
            
            for field in key_fields:
                if item.get(field) and str(item[field]).strip() and item[field] != '':
                    non_empty_count += 1
            
            # Keep record if at least 2 out of 5 key fields have data (adjusted threshold)
            if non_empty_count >= 2:
                cleaned_data.append(item)
        
        print(f"🧹 Data cleaning: {len(data)} → {len(cleaned_data)} records (removed {len(data) - len(cleaned_data)} low-quality entries)")
        return cleaned_data
    
    def save_to_files(self, data, city):
        """
        Enhanced save function with both CSV and Excel output
        """
        if not data:
            print("❌ No data to save!")
            return
        
        # Clean the data first
        cleaned_data = self.clean_data(data)
        
        if not cleaned_data:
            print("❌ No valid data remaining after cleaning!")
            return
        
        # Create DataFrame
        df = pd.DataFrame(cleaned_data)
        
        # Remove duplicate entries
        df = df.drop_duplicates(subset=['Name', 'Address'], keep='first')
        
        # Replace empty values with empty strings for better display
        df = df.fillna('')
        
        # Sort by business type and rating (handle empty ratings)
        df['Rating_Sort'] = pd.to_numeric(df['Rating'], errors='coerce').fillna(0)
        df = df.sort_values(['Business_Type', 'Rating_Sort'], ascending=[True, False])
        df = df.drop(columns=['Rating_Sort'])  # Remove helper column
        
        # Create filename with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        base_filename = f"{city.replace(' ', '_')}_businesses_{timestamp}"
        
        # Create summary data
        summary_data = []
        for business_type in self.business_types.keys():
            business_df = df[df['Business_Type'] == business_type]
            if not business_df.empty:
                # Handle rating calculation safely
                numeric_ratings = pd.to_numeric(business_df['Rating'], errors='coerce')
                avg_rating = numeric_ratings.mean()
                
                # Count entries with phone numbers and websites
                phone_count = len(business_df[business_df['Phone'].astype(str).str.strip() != ''])
                website_count = len(business_df[business_df['Website'].astype(str).str.strip() != ''])
                
                summary_data.append({
                    'Business_Type': business_type,
                    'Total_Count': len(business_df),
                    'With_Phone': phone_count,
                    'With_Website': website_count,
                    'Phone_Percentage': f"{round((phone_count/len(business_df))*100, 1)}%",
                    'Website_Percentage': f"{round((website_count/len(business_df))*100, 1)}%",
                    'Avg_Rating': round(avg_rating, 2) if not pd.isna(avg_rating) else 'No Ratings'
                })
        
        summary_df = pd.DataFrame(summary_data) if summary_data else pd.DataFrame()
        
        # === CSV OUTPUT ===
        print(f"\n💾 Saving CSV files...")
        
        # Save main CSV file
        main_csv_filename = f"{base_filename}_all_data.csv"
        df.to_csv(main_csv_filename, index=False, encoding='utf-8-sig')
        
        # Save summary CSV
        if not summary_df.empty:
            summary_csv_filename = f"{base_filename}_summary.csv"
            summary_df.to_csv(summary_csv_filename, index=False, encoding='utf-8-sig')
        
        # Create separate CSV files for each business type
        csv_business_files = []
        for business_type in self.business_types.keys():
            business_df = df[df['Business_Type'] == business_type]
            if not business_df.empty:
                clean_name = business_type.replace(' ', '_').replace('&', 'and').replace('/', '_')
                business_csv_filename = f"{base_filename}_{clean_name}.csv"
                business_df.to_csv(business_csv_filename, index=False, encoding='utf-8-sig')
                csv_business_files.append(business_csv_filename)
        
        # === EXCEL OUTPUT ===
        print(f"📊 Saving Excel file...")
        
        try:
            # Save single Excel file with multiple sheets
            main_excel_filename = f"{base_filename}_complete.xlsx"
            
            with pd.ExcelWriter(main_excel_filename, engine='openpyxl') as writer:
                # Sheet 1: All data combined
                df.to_excel(writer, sheet_name='All_Businesses', index=False)
                
                # Sheet 2: Summary sheet
                if not summary_df.empty:
                    summary_df.to_excel(writer, sheet_name='Summary', index=False)
                
                # Individual business type sheets
                for business_type in self.business_types.keys():
                    business_df = df[df['Business_Type'] == business_type]
                    if not business_df.empty:
                        # Clean sheet name (Excel has restrictions)
                        clean_sheet_name = business_type.replace(' ', '_').replace('&', 'and').replace('/', '_')
                        # Truncate if too long (Excel sheet name limit is 31 characters)
                        if len(clean_sheet_name) > 31:
                            clean_sheet_name = clean_sheet_name[:28] + "..."
                        
                        business_df.to_excel(writer, sheet_name=clean_sheet_name, index=False)
            
            excel_success = True
            excel_business_files = []  # No separate files needed
            
        except ImportError:
            print("⚠️  Excel export requires 'openpyxl' package. Install with: pip install openpyxl")
            excel_success = False
            main_excel_filename = "Not created (missing openpyxl)"
        except Exception as e:
            print(f"⚠️  Error creating Excel file: {e}")
            excel_success = False
            main_excel_filename = "Error during creation"
        
        # === OUTPUT SUMMARY ===
        print(f"\n✅ Data saved successfully in both formats!")
        print("=" * 80)
        
        print(f"📄 CSV FILES:")
        print(f"    📊 Main file: {main_csv_filename}")
        if not summary_df.empty:
            print(f"    📈 Summary file: {summary_csv_filename}")
        print(f"    📁 Individual category files: {len(csv_business_files)} files created")
        
        print(f"\n📊 EXCEL FILE:")
        if excel_success:
            print(f"    📗 Complete workbook: {main_excel_filename}")
            print(f"    📑 Contains {len([bt for bt in self.business_types.keys() if len(df[df['Business_Type'] == bt]) > 0]) + 2} sheets:")
            print(f"        • All_Businesses (combined data)")
            print(f"        • Summary (statistics)")
            for business_type in self.business_types.keys():
                if len(df[df['Business_Type'] == business_type]) > 0:
                    clean_name = business_type.replace(' ', '_').replace('&', 'and').replace('/', '_')
                    if len(clean_name) > 31:
                        clean_name = clean_name[:28] + "..."
                    print(f"        • {clean_name}")
        else:
            print(f"    ❌ Excel file: {main_excel_filename}")
            print(f"    💡 To enable Excel export: pip install openpyxl")
        
        print(f"\n📈 DATA SUMMARY:")
        print(f"    🏢 Total unique records: {len(df)}")
        print(f"    📋 Business categories: {len([bt for bt in self.business_types.keys() if len(df[df['Business_Type'] == bt]) > 0])}")
        
        # Enhanced summary with insights
        print(f"\n📊 Detailed Summary for {city}:")
        print("=" * 80)
        
        total_with_phone = 0
        total_with_website = 0
        total_businesses = len(df)
        
        for business_type in self.business_types.keys():
            business_df = df[df['Business_Type'] == business_type]
            if not business_df.empty:
                # Safe rating calculation
                numeric_ratings = pd.to_numeric(business_df['Rating'], errors='coerce')
                avg_rating = numeric_ratings.mean()
                
                # Count contact info availability
                phone_count = len(business_df[business_df['Phone'].astype(str).str.strip() != ''])
                website_count = len(business_df[business_df['Website'].astype(str).str.strip() != ''])
                
                total_with_phone += phone_count
                total_with_website += website_count
                
                phone_pct = round((phone_count/len(business_df))*100, 1)
                website_pct = round((website_count/len(business_df))*100, 1)
                
                print(f"🏢 {business_type}:")
                print(f"    📍 Total Locations: {len(business_df)}")
                print(f"    📞 With Phone: {phone_count} ({phone_pct}%)")
                print(f"    🌐 With Website: {website_count} ({website_pct}%)")
                print(f"    ⭐ Avg Rating: {round(avg_rating, 2) if not pd.isna(avg_rating) else 'No Ratings'}")
                print("-" * 60)
        
        # Overall statistics
        print(f"📈 OVERALL STATISTICS:")
        print(f"    🏢 Total Businesses: {total_businesses}")
        if total_businesses > 0:
            print(f"    📞 Total with Phone: {total_with_phone} ({round((total_with_phone/total_businesses)*100, 1)}%)")
            print(f"    🌐 Total with Website: {total_with_website} ({round((total_with_website/total_businesses)*100, 1)}%)")
        else:
            print("    No businesses found to calculate overall statistics.")
        print("=" * 80)
        
        print(f"\n💡 File Usage Tips:")
        print("    CSV Files:")
        print("      • Compatible with Excel, Google Sheets, and any spreadsheet software")
        print("      • Best for data analysis and importing into other tools")
        print("      • UTF-8 encoding supports international characters")
        print("    Excel File:")
        print("      • Single workbook with multiple organized sheets")
        print("      • 'All_Businesses' sheet contains complete combined data")
        print("      • Individual category sheets for focused analysis")
        print("      • 'Summary' sheet with statistics and insights")
        print("      • Better formatting and easier navigation")
    
    def run_interactive(self):
        """
        Enhanced interactive version with dual format output
        """
        print("=" * 70)
        print("🏢 ENHANCED BUSINESS LOCATOR - CSV & EXCEL OUTPUT (Places API New)")
        print("=" * 70)
        print("📍 This tool will find the following business types:")
        
        for idx, business_type in enumerate(self.business_types.keys(), 1):
            print(f"    {idx:2d}. {business_type}")
        
        print("=" * 70)
        print("💡 Features:")
        print("    ✅ Phone numbers and websites included")
        print("    ✅ Output in both CSV and Excel formats")
        print("    ✅ Single Excel file with multiple organized sheets")
        print("    ✅ Combined data sheet + individual category sheets")
        print("    ✅ Summary statistics and percentages")
        print("    ✅ Individual CSV files for each business category")
        print("=" * 70)
        print("⚠️  Important Notes:")
        print("    • Output: CSV files + One comprehensive Excel file")
        print("    • Excel file contains multiple sheets (combined + individual categories)")
        print("    • Excel requires 'openpyxl' package: pip install openpyxl")
        print("    • Estimated time: 5-10 minutes for larger cities")
        print("    • Files saved with UTF-8 encoding")
        print("    • This version uses 'Places API (New)'. Ensure it's enabled in Google Cloud Console.")
        print("=" * 70)
        
        while True:
            try:
                city = input("\n🏙️  Enter city name (or 'quit' to exit): ").strip()
                
                if city.lower() in ['quit', 'exit', 'q']:
                    print("👋 Thank you for using Enhanced Business Locator!")
                    break
                
                if not city:
                    print("⚠️  Please enter a valid city name.")
                    continue
                
                # Confirmation for large searches
                print(f"\n🚀 Starting enhanced search for {city}...")
                print(f"⏱️  Estimated time: 5-10 minutes (fetching detailed data for {len(self.business_types)} categories)")
                print("💰 This will make multiple detailed API calls (ensure billing is enabled)")
                print("📄 Output: CSV files + One comprehensive Excel workbook")
                print("📑 Excel file will have combined data + individual category sheets")
                
                confirm = input("Continue? (y/n): ").strip().lower()
                if confirm not in ['y', 'yes']:
                    continue
                
                # Record start time
                start_time = time.time()
                
                # Get all business data
                all_data = self.get_all_businesses(city)
                
                # Calculate execution time
                end_time = time.time()
                execution_time = round(end_time - start_time, 2)
                
                if all_data:
                    print(f"\n⏱️  Search completed in {execution_time} seconds")
                    
                    # Save the data in both formats
                    self.save_to_files(all_data, city)
                    
                    # Ask if user wants to continue
                    print("\n" + "="*60)
                    continue_choice = input("🔄 Search another city? (y/n): ").strip().lower()
                    if continue_choice not in ['y', 'yes']:
                        print("👋 Thank you for using Enhanced Business Locator!")
                        break
                else:
                    print(f"❌ No businesses found for {city}. Please try another city or refine search terms.")
                    
            except KeyboardInterrupt:
                print("\n\n👋 Program interrupted. Goodbye!")
                break
            except Exception as e:
                print(f"❌ An unexpected error occurred: {e}")
                print("Please try again or contact support if the issue persists.")
                continue

def main():
    """
    Enhanced main function with dual format setup instructions, guiding the user
    through Google Cloud Console prerequisites if the API key fails validation.
    """
    print("🔑 Enhanced Google Maps API Business Locator - CSV & Excel Output")
    print("=" * 70)
    
    # Check for required packages
    try:
        import openpyxl
        excel_available = True
        print("✅ Excel support: Available (openpyxl found)")
    except ImportError:
        excel_available = False
        print("⚠️  Excel support: Limited (install openpyxl for full Excel support)")
        print("    Run: pip install openpyxl")
    
    print("\n--- API Key Setup Guide ---")
    print("To use this tool, you need a Google Maps API Key with specific APIs enabled.")
    print("If you encounter 'REQUEST_DENIED' errors, please follow these steps carefully:")
    print("===================================================================")
    print("1. 🌐  Go to Google Cloud Console: https://console.cloud.google.com")
    print("2. 📁  Select or Create a Project. (Top left dropdown menu)")
    print("3. 💳  Enable Billing for your project. (Required for all Places API usage)")
    print("4. 🔧  Enable Necessary APIs:")
    print("      Navigate to 'APIs & Services' > 'Enabled APIs & Services'.")
    print("      Click '+ ENABLE APIS AND SERVICES' and search for & enable:")
    print("      •  'Places API' (This is crucial for the script to work)")
    print("5. 🔑  Create or Get Your API Key:")
    print("      Go to 'APIs & Services' > 'Credentials'.")
    print("      Create a new API key or copy an existing one.")
    print("6. 🔒  Restrict Your API Key (Highly Recommended for security):")
    print("      Edit your API key and under 'API restrictions', select 'Restrict key' to 'Places API'.")
    print("===================================================================")
    print("Once these steps are done, enter your API key below.")

    api_key = input("\nEnter your Google Maps API key: ").strip()
    
    if not api_key:
        print("\n❌ API key is required to use this tool. Please restart and provide it.")
        return
    
    try:
        print("\n🔄 Initializing and validating API key...")
        
        # Initialize the locator
        locator = BusinessLocator(api_key)
        
        # Test the API key with a simple query using the new find_place method
        # This will use the "Find Place From Text" (New) endpoint
        test_result = locator.gmaps.find_place(
            input="test restaurant in New York",
            input_type='textquery',
            fields=['place_id'] # Request minimal fields for a quick test
        )
        if 'candidates' in test_result and test_result['candidates']:
            print("✅ API key validated successfully using Places API (New)!")
        else:
            # If no candidates found, it might still be valid but with no matching results
            # or some other issue. Guide the user back to the setup.
            print("⚠️  API key validation inconclusive. This could mean:")
            print("    - No test results found (which is rare but possible).")
            print("    - The API key is not fully configured or has incorrect permissions.")
            print("    Please double-check the API Key Setup Guide above.")
            return # Exit if validation is inconclusive to prevent further errors
        
        # Show dual output benefits
        print("\n📄 Dual Format Output Benefits:")
        print("    CSV Files:")
        print("      ✅ Universal compatibility (Excel, Google Sheets, etc.)")
        print("      ✅ Easy data import for analysis tools")
        print("      ✅ Lightweight and fast to open")
        print("    Excel File:")
        print("      ✅ Single comprehensive workbook with multiple sheets")
        print("      ✅ 'All_Businesses' sheet with complete combined data")
        print("      ✅ Individual category sheets for focused analysis")
        print("      ✅ 'Summary' sheet with statistics and insights")
        print("      ✅ Better formatting and presentation")
        
        proceed = input(f"\nProceed with dual-format output? (y/n): ").strip().lower()
        if proceed not in ['y', 'yes']:
            print("👋 Exiting. Thank you!")
            return
        
        # Run the interactive locator
        locator.run_interactive()
        
    except googlemaps.exceptions.ApiError as e:
        print(f"\n❌ API Error: {e}")
        print("📋 This error often means your API key isn't fully set up.")
        print("   Please review the 'API Key Setup Guide' provided above.")
        print(f"   Specific error details: {e}")
    except Exception as e:
        print(f"\n❌ An unexpected error occurred during API key initialization or validation: {e}")
        print("   Please ensure you have an active internet connection and your API key is correctly entered.")

if __name__ == "__main__":
    # Display package requirements
    print("📦 Required packages:")
    print("    Essential: googlemaps, pandas")
    print("    For Excel: openpyxl")
    print("💻 Install/Upgrade with: pip install --upgrade googlemaps pandas openpyxl") # Added upgrade for googlemaps
    print("=" * 70)
    
    main()

📦 Required packages:
    Essential: googlemaps, pandas
    For Excel: openpyxl
💻 Install/Upgrade with: pip install --upgrade googlemaps pandas openpyxl
🔑 Enhanced Google Maps API Business Locator - CSV & Excel Output
✅ Excel support: Available (openpyxl found)

--- API Key Setup Guide ---
To use this tool, you need a Google Maps API Key with specific APIs enabled.
If you encounter 'REQUEST_DENIED' errors, please follow these steps carefully:
1. 🌐  Go to Google Cloud Console: https://console.cloud.google.com
2. 📁  Select or Create a Project. (Top left dropdown menu)
3. 💳  Enable Billing for your project. (Required for all Places API usage)
4. 🔧  Enable Necessary APIs:
      Navigate to 'APIs & Services' > 'Enabled APIs & Services'.
      Click '+ ENABLE APIS AND SERVICES' and search for & enable:
      •  'Places API' (This is crucial for the script to work)
5. 🔑  Create or Get Your API Key:
      Go to 'APIs & Services' > 'Credentials'.
      Create a new API key or copy an existing on